In [1]:
# Import necessary libraries
import re
import httpx
from enum import Enum


# Define the ProviderType enum
class ProviderType(Enum):
    APP = "APP"
    GAME = "GAME"

In [2]:
# QQDownloaderProvider implementation

with httpx.Client() as client:
    response = client.get("https://sj.qq.com/app")
    cookie_header = response.headers.get("set-cookie", "")
    match = re.search(r"YYB_HOME_UUID=([^;]+);", cookie_header)

    assert match, "Failed to fetch cookie"

uin = match[1]
uin

'eb80313b-dfb1-496a-a67a-cfa07b01cdff'

In [3]:
def retrieve(page: int, provider_type: ProviderType = ProviderType.APP):
    headers = {
        "Cache-Control": "no-cache",
        "Content-Type": "text/plain;charset=UTF-8",
        "Referer": "https://sj.qq.com/app",
    }

    payload = {
        "head": {
            "cmd": "dynamicard_yybhome",
            "authInfo": {"businessId": "AuthName"},
            "deviceInfo": {"platformType": 2, "platform": 2},
            "userInfo": {"guid": uin},
            "expSceneIds": "",
            "hostAppInfo": {
                "scene": {
                    ProviderType.APP: "app_center",
                    ProviderType.GAME: "game_center",
                }[provider_type]
            },
        },
        "body": {
            "bid": "yybhome",
            "offset": 0,
            "size": 10,
            "preview": False,
            "listS": {"region": {"repStr": ["CN"]}, "cate_alias": {"repStr": ["all"]}},
            "listI": {"limit": {"repInt": [24]}, "offset": {"repInt": [page]}},
            "layout": {
                ProviderType.APP: "YYB_HOME_APP_LIBRARY_LIST",
                ProviderType.GAME: "YYB_HOME_GAME_LIBRARY_LIST_ALGRITHM",
            }[provider_type],
        },
    }

    with httpx.Client() as client:
        response = client.post(
            "https://yybadaccess.3g.qq.com/v2/dynamicard_yybhome",
            headers=headers,
            json=payload,
        )

    assert response.status_code == 200, "Failed to fetch data"
    response_data = response.json()
    ret = response_data.get("ret")
    data = response_data.get("data")
    assert ret == 0, "Failed to fetch data"

    return [item for item in data["components"][0]["data"]["itemData"]]


retrieve(1)

[{'pkg_name': 'com.qzone',
  'app_id': '9959',
  'name': 'QQ空间',
  'icon': 'http://pp.myapp.com/ma_icon/0/icon_9959_1724817157/256',
  'md_5': 'F623052C7BC882842CFB4271C21C76C4',
  'download_url': 'http://imtt2.dd.qq.com/sjy.00009/sjy.00004/16891/apk/F623052C7BC882842CFB4271C21C76C4.apk?fsname=com.qzone_8.9.1.288.apk',
  'is_game': '0',
  'is_cloud_game': '0',
  'channel_info': '',
  'apk_size': '77280398',
  'ios_url': 'https://itunes.apple.com/cn/app/id364183992&mt=8',
  'developer': '腾讯数码（天津）有限公司',
  'update_time': '1729245079',
  'privacy_agreement': 'https://cftweb.3g.qq.com/privacy/privacyPolicy?content_id=f6da60aeeb725db6cd9fa802fdbdf6a7',
  'version_name': '8.9.1.288',
  'operator': '深圳市腾讯计算机系统有限公司',
  'game_type': '0',
  'is_pc_yyb_available': '1',
  'is_booking': '0',
  'report_info': {'cid': '',
   'ctype': '',
   'trace_id': '472c5a72-f8ff-11ef-aac5-5254001eb2b1',
   'slot': 1,
   'screenorder': 1,
   'rid': '',
   'appid': '9959',
   'pkgname': 'com.qzone',
   'appname': '

In [4]:
import aria2p

# initialization, these are the default values
aria2 = aria2p.API(aria2p.Client(host="http://localhost", port=16800, secret=""))

[*aria2.get_downloads()]

[]

In [5]:
import time
from tqdm.notebook import tqdm
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import hashlib

# [name, pkg_name, apk_filename]
columns = []
apk_filepath = Path("/run/media/mix/64B2AAFCB2AAD1BA/ExAPP") / "apks"
apk_filepath.mkdir(parents=True, exist_ok=True)

pool = ThreadPoolExecutor()


def check_apk_integrity(app):
    download_path = apk_filepath / "{app_id}_{pkg_name}_{version_name}.apk".format_map(
        app
    )

    if not download_path.is_file():
        return False

    md5 = hashlib.md5()
    with (
        download_path.open("rb", buffering=16 * 1024 * 1024) as f,
        tqdm(
            total=int(app["apk_size"]),
            desc="Verifying {name} {pkg_name}".format_map(app),
            unit="B",
            unit_scale=True,
            leave=False,
        ) as pbar,
    ):
        for chunk in f:
            md5.update(chunk)
            pbar.update(len(chunk))

    return md5.digest() == bytes.fromhex(app["md_5"])


def download_apk(app):
    download_path = apk_filepath / "{app_id}_{pkg_name}_{version_name}.apk".format_map(
        app
    )

    if check_apk_integrity(app):
        return app, download_path

    # Add download to aria2
    download = aria2.add_uris(
        [app["download_url"]],
        options={
            "dir": str(apk_filepath.absolute()),
            "out": download_path.name,
            "continue": "true",
        },
    )

    # Wait for download to complete with progress display
    with tqdm(
        total=int(app["apk_size"]),
        desc=f"Downloading {app['name']}",
        unit="B",
        unit_scale=True,
        leave=False,
    ) as pbar:
        while not download.is_complete:
            download.update()
            # Update progress bar based on completed length
            pbar.n = download.completed_length
            pbar.refresh()
            time.sleep(0.5)

    if not check_apk_integrity(app):
        download_path.unlink()
        return app, None

    return app, download_path


apps = [app for page in tqdm(range(1, 100)) for app in retrieve(page)]
for app, download_path in tqdm(
    pool.map(download_apk, apps),
    total=len(apps),
):
    if download_path is None:
        print(f"Failed to download {app['name']}")
        continue
    columns.append([app["name"], app["pkg_name"], str(download_path)])

  0%|          | 0/99 [00:00<?, ?it/s]

Verifying 酷我音乐 cn.kuwo.player:   0%|          | 0.00/179M [00:00<?, ?B/s]

Verifying 京东 com.jingdong.app.mall:   0%|          | 0.00/106M [00:00<?, ?B/s]

Verifying 唯品会 com.achievo.vipshop:   0%|          | 0.00/76.3M [00:00<?, ?B/s]

Verifying 360手机助手 com.qihoo.appstore:   0%|          | 0.00/27.6M [00:00<?, ?B/s]

...

Verifying 蜂鸟部落 com.fengniaobuluocps:   0%|          | 0.00/72.0M [00:00<?, ?B/s]

Verifying 极米 com.xgimi.zhushou:   0%|          | 0.00/149M [00:00<?, ?B/s]

Verifying 羊驼韩语 com.ytekorean.client:   0%|          | 0.00/156M [00:00<?, ?B/s]

Verifying 小熊美术 com.meishubao.artaiclass:   0%|          | 0.00/333M [00:00<?, ?B/s]

Verifying 腾讯文档 com.tencent.docs:   0%|          | 0.00/295M [00:00<?, ?B/s]

In [6]:
from pandas import DataFrame

df = DataFrame(columns, columns=["name", "pkg_name", "path"])
df.to_csv(apk_filepath.parent / "apps.csv", index=False)
